# Bzdawka_Igor_assignment_2
#### Igor Bzdawka
#### 2023-01-05

# Homework Assignment 2 - Report
## Introduction
This report is a summary of the homework assignment nr 2. This assignment was focused on learning the Python *pandas* package, as well as revising the *numpy* functions we learned during the semester. As the main task, five SQL queries, the same as in the first assignment, were given to be implemented using **_pandas_** structure, accompanied by **_numpy_** functions.

The data that the queries read from come from the website <https://travel.stackexchange.com/>.
The data frames given are:

* Badges.csv.gz,
* Comments.csv.gz,
* Posts.csv.gz,
* Users.csv.gz,
* Votes.csv.gz.

## Setup

To be able to see the results of the functions, we first need to refer to the Python file containing the functions returning the results of the queries, as follows:

In [1]:
from Bzdawka_Igor_assignment_2 import *

Obviously, this Jupyter Notebook needs to be in the same directory as the Python file in order for it to be able to be seen.


We also need to provide the *pandas* and *numpy* packages, so that the functions can use them to resolve the functions:

In [2]:
import numpy as np
import pandas as pd

Finally, we need to set up the database, which will allow the connections between the tables and calculating the queries.

The output of the *pandas* function __*read_sql_query()*__ for each query will serve as the **reference solution**, as it simply requires pasting the SQL code as the argument. In order to evaluate it though, we first need to set up the database, which will figure as an environment which the function will use to work on.

The code below provides a database and evaluates all the reference solutions for each query, each one named __*ref_n*__, where *n* stands for the number of the query:

In [3]:
# Importing libraries for setting up the database
import os, os.path
import sqlite3
import tempfile

# Reading the packages and saving them as pandas data frames
Badges = pd.read_csv("Badges.csv.gz", compression="gzip")
Users = pd.read_csv("Users.csv.gz", compression="gzip")
Votes = pd.read_csv("Votes.csv.gz", compression="gzip")
Comments = pd.read_csv("Comments.csv.gz", compression="gzip")
Posts = pd.read_csv("Posts.csv.gz", compression="gzip")

# Setting up the database
baza = os.path.join(tempfile.mkdtemp(), 'example.db')
if os.path.isfile(baza):
    os.remove(baza)
# Creating connection
conn = sqlite3.connect(baza)

# Appending the packages via the connection to the database
Badges.to_sql("Badges", conn)
Comments.to_sql("Comments", conn)
Posts.to_sql("Posts", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)

# Evaluating the reference solutions
ref_1 = pd.read_sql_query('''SELECT STRFTIME('%Y', CreationDate) AS Year, COUNT(*) AS TotalNumber
FROM Posts
GROUP BY Year''',conn)

ref_2 = pd.read_sql_query('''SELECT Id, DisplayName, SUM(ViewCount) AS TotalViews
FROM Users
JOIN (
SELECT OwnerUserId, ViewCount FROM Posts WHERE PostTypeId = 1
) AS Questions
ON Users.Id = Questions.OwnerUserId
GROUP BY Id
ORDER BY TotalViews DESC
LIMIT 10''',conn)

ref_3 = pd.read_sql_query('''SELECT Year, Name, MAX((Count * 1.0) / CountTotal) AS MaxPercentage
FROM (
SELECT BadgesNames.Year, BadgesNames.Name, BadgesNames.Count, BadgesYearly.CountTotal
FROM (
SELECT Name, COUNT(*) AS Count, STRFTIME('%Y', Badges.Date) AS Year
FROM Badges
GROUP BY Name, Year
) AS BadgesNames
JOIN (
SELECT COUNT(*) AS CountTotal, STRFTIME('%Y', Badges.Date) AS Year
FROM Badges
GROUP BY YEAR
) AS BadgesYearly
ON BadgesNames.Year = BadgesYearly.Year
)
GROUP BY Year''',conn)

ref_4 = pd.read_sql_query('''SELECT Title, CommentCount, ViewCount, CommentsTotalScore, DisplayName, Reputation, Location
FROM (
SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount,
CmtTotScr.CommentsTotalScore
FROM (
SELECT PostId, SUM(Score) AS CommentsTotalScore
FROM Comments
GROUP BY PostId
) AS CmtTotScr
JOIN Posts ON Posts.Id = CmtTotScr.PostId
WHERE Posts.PostTypeId=1
) AS PostsBestComments
JOIN Users ON PostsBestComments.OwnerUserId = Users.Id
ORDER BY CommentsTotalScore DESC
LIMIT 10''',conn)

ref_5 = pd.read_sql_query('''SELECT Posts.Title, STRFTIME('%Y-%m-%d', Posts.CreationDate) AS Date, VotesByAge.*
FROM Posts
JOIN (
SELECT PostId,
MAX(CASE WHEN VoteDate = 'before' THEN Total ELSE 0 END) BeforeCOVIDVotes,
MAX(CASE WHEN VoteDate = 'during' THEN Total ELSE 0 END) DuringCOVIDVotes,
MAX(CASE WHEN VoteDate = 'after' THEN Total ELSE 0 END) AfterCOVIDVotes,
SUM(Total) AS Votes
FROM (
SELECT PostId,
CASE STRFTIME('%Y', CreationDate)
WHEN '2022' THEN 'after'
WHEN '2021' THEN 'during'
WHEN '2020' THEN 'during'
WHEN '2019' THEN 'during'
ELSE 'before'
END VoteDate, COUNT(*) AS Total
FROM Votes
WHERE VoteTypeId IN (3, 4, 12)
GROUP BY PostId, VoteDate
) AS VotesDates
GROUP BY VotesDates.PostId
) AS VotesByAge ON Posts.Id = VotesByAge.PostId
WHERE Title NOT IN ('') AND DuringCOVIDVotes > 0
ORDER BY DuringCOVIDVotes DESC, Votes DESC
LIMIT 20''',conn)

# Closing the connection
conn.close()

Naturally, the code above will work only provided the packages are in the same directory as this file.

## Solutions analysis

To confirm that the result of the *pandas* function and the reference solution are the same, the pandas function __*.equals()*__ will be applied on both of them. If the output of this function is __*True*__, it means that the output of the function is equal to the reference solution, proving the correct implementation.

### Query 1

#### Short Summary
The first query returns the total number of posts for each year. The output is grouped by year in ascending order.

#### Output Correctness Comparison
The variable **_sol_n_**, where *n* stands for the number of the query, will contain the result of computing the *n*-th query using *pandas* functions. Later, it shall be compared to the **_ref_n_** using __*.equals()*__ function, as below. The value __*True*__ as a result of the comparison will be a proof of the correct implementation of the query.

In [4]:
sol_1 = solution_1(Posts)
print(sol_1.equals(ref_1))

True


As we can see by the result of the code chunk above, the solution is equal to the reference solution, which proves the accuracy and correctness of the implementation of the query.

### Query 2

#### Short Summary
This query represents the scoreboard, which displays the top 10 users which posts' sum of views was the greatest. The scoreboard includes the *ID* of the user, their *name* on the website and *total number of views* of all the user's posts.

#### Output Correctness Comparison

In [5]:
sol_2 = solution_2(Users, Posts)
print(sol_2.equals(ref_2))

True


As we can see, the result proves that the implementation was correct.

### Query 3

#### Short Summary
The third query returns a table showing, for each year, the name of the badge that was most frequently given, along with its frequency fraction - the number of times it was given, divided by the number of all badges given in this year, expressed as **_MaxPercentage_**.

#### Output Correctness Comparison

In [6]:
sol_3 = solution_3(Badges)
print(sol_3.equals(ref_3))

True


As we can see, the result proves that the implementation was correct.

### Query 4

#### Short Summary
The query no. 4 returns a table, which presents 10 posts from the website, which total score gained by all the comments under them was the biggest. The table provides the **_titles_** of the posts, the **_number of comments_** under each post, the **_total view count_** of each post, the **_total score of comments_** mentioned above, the **_name_** of the posting user, their **_reputation_** score and, if given, their **_location_**.


#### Output Correctness Comparison



In [7]:
sol_4 = solution_4(Comments, Posts, Users)
print(sol_4.equals(ref_4))

True


As we can see, the result proves that the implementation was correct.

### Query 5

#### Short Summary
The last query gives 20 rows of result. The output table is the top 20 posts, ordered by the number of votes they got during the **COVID pandemic**, which was a period of time between year **2019 and 2021**. The posts shown in the scoreboard were filtered, so that the only posts taken into consideration were those which **votes score** is a **positive number**, and those which **title** is **non-empty**. Additionally, the only votes taken into account were those which **VoteTypeId** was equal to either 3, 4 or 12. The table presented in output includes the __*title*__ of the post, the __*full (YYYY-MM-DD)*__ date of posting it, the post __*ID number*__, as well as __*votes scoreboards*__, divided in columns on votes given __*before, during and after*__ the pandemic. Additionally, a __*total votes score*__ for each post was provided as the last column.



#### Output Correctness Comparison



In [8]:
sol_5 = solution_5(Posts, Votes)
print(sol_5.equals(ref_5))

True


As we can see, the result proves that the implementation was correct.

## Conclusions

The *pandas* package allows for a very smooth and intuitive processing data frames. The ability to chain commands one after another, similarly to piping in *dplyr*, provides the same instructions in smaller space, also making the code more readable and comprehensive. Moreover, fulfilling this project was a valuable experience gained in the field of Python programming.